In [11]:
import scipy as sp 
from matplotlib import pyplot as plt
import cupy as cp
from cupy.linalg import eigh

from tqdm import *
import time
import pennylane as qml
import pennylane.numpy as np 

In [6]:
def generate_Heisenberg_hamiltonian(n):
    """
    Generating Hamiltonian of n-qubit Heisenberg model.

    Args:
        - n (int): The number of qubits 

    Returns:
        - cost_h (qml.Hamiltonian): the hamiltonian of n-qubit Heisenberg model
    """
    coeffs = [1]*3*(n-1)
    obs = []
    for i in range(n-1):
        obs.append(qml.PauliX(i) @ qml.PauliX(i+1) )
        obs.append(qml.PauliY(i) @ qml.PauliY(i+1) )
        obs.append(qml.PauliZ(i) @ qml.PauliZ(i+1) )
    cost_h = qml.Hamiltonian(coeffs, obs)
    return cost_h

In [7]:
def ground_energy(n):
    """ 
    Computing the ground state energy (minimum eigenvalue) of the n-qubit Heisenberg model using CuPy.
    
    Args:
        - n (int): The number of qubits. 

    Returns:
        - min_energy (floats): Minimum eigenvalue of the Heisenberg model.
    
    """
    hamiltonian = generate_Heisenberg_hamiltonian(n)
    hamiltonian_matrix = qml.matrix(hamiltonian)
    h_matrix_cupy = cp.array(hamiltonian_matrix)
    eigv,eigc = eigh(h_matrix_cupy)

    return eigv[0]
    

In [8]:
def qnn_model(params,depth,cost_h):
    def U3_layer(params):
        for i in range(n_wires):
            qml.U3(params[3*i],params[3*i+1],params[3*i+2],wires=i)
    
    def IsingXYZ_layer(params):
        for i in range(n_wires-1):
            qml.IsingXY(params[3*i],wires=[i,(i+1)%n_wires])
            qml.IsingXY(params[3*i+1],wires=[i,(i+1)%n_wires])
            qml.IsingZZ(params[3*i+2],wires=[i,(i+1)%n_wires])
            pass

    def U3_IsingXYZ(params):
        U3_layer(params[0])
        IsingXYZ_layer(params[1])

    qml.layer(U3_IsingXYZ,depth,params[:-1])
    
    return qml.expval(cost_h)

In [15]:
n = 20
depth = 4
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.2)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 200
params = init_params
start_time = time.time()
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        end_time = time.time()
        cost_tmp = cost_fn(params)
        duration = (end_time - start_time)
        if( i % 10 == 0):
            print("Iter: ", i+1, " | ", "energy value is ", cost_tmp, "  the duration is ", duration, "s")

Iter:  1  |  energy value is  -1.9172185018786956   the duration is  23.02931785583496 s
Iter:  11  |  energy value is  -27.516576583051418   the duration is  310.25386238098145 s
Iter:  21  |  energy value is  -31.228787996286773   the duration is  599.3446316719055 s
Iter:  31  |  energy value is  -32.77712500444348   the duration is  893.2062692642212 s
Iter:  41  |  energy value is  -33.27422449383771   the duration is  1183.390209197998 s
Iter:  51  |  energy value is  -33.51450242156993   the duration is  1473.6073241233826 s
Iter:  61  |  energy value is  -33.62326998458488   the duration is  1770.2943587303162 s
Iter:  71  |  energy value is  -33.68081782566351   the duration is  2077.7988543510437 s
Iter:  81  |  energy value is  -33.733707171893634   the duration is  2385.907058238983 s
Iter:  91  |  energy value is  -33.80190565778025   the duration is  2688.3033590316772 s
Iter:  101  |  energy value is  -33.875227533089124   the duration is  2994.214384317398 s
Iter:  111 

In [ ]:
-33.733707171893634